# AlphaPose

AlphaPose combines many recent works on Multi-person pose estimation and tracking, available at: https://github.com/MVIG-SJTU/AlphaPose

Within this notebook, we'll be using AlphaPose to perform large-scale multi-person (Crowd) pose estimation on a desired input dataset of images.

**Instructions:**

You need to follow these instructions carefully to get this notebook to work as intended. Perform the following steps:

1.   Install the required dependencies into this Colab instance - once this is done, **restart** the runtime using the tab-menu at the top.

2.   Once restarted, you can then run the next series of cells to import required libraries, clone the AlphaPose repo, revert to a previous repo version (due to a current system-breaking bug introduced recently), and build AlphaPose. This will take approx 5 minutes.

3.   Download the pre-trained models and weights for AlphaPose. To do this you'll need to authenticate your Google account as prompted.

4.   Create `input_images` and `output_images` directories, and then upload custom dataset to `input_images` folder.

5.   Perform extraction of pose features from all input images using AlphaPose.

6.   Save results as a .zip and download.

## 1. Install Dependencies

AlphaPose currently requires specific versions of PyTorch, Torchvision and some other well-used libraries in order to work without issues. We'll install these below.

Once you've done this, you need to restart the runtime (using the runtime tab at the top).

In [1]:
# dependencies for AlphaPose - install and restart runtime
!pip install pyyaml==5.2
!pip install scipy==1.1.0
!pip install torch==1.2.0 torchvision==0.4.0
!pip install pillow==6.2.2

     |████████████████████████████████| 265 kB 5.2 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.2-cp37-cp37m-linux_x86_64.whl size=44225 sha256=b17fc78acb22df02f72760d641d16619b70481d03422ba16a671de4110a4e703
  Stored in directory: /root/.cache/pip/wheels/c5/67/65/80b02387630c3794fc9bab68ca0de3dd87d639d65936026527
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 31.2 MB 144 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0,

## 2. Import dependencies, clone AlphaPose repo, and install from source.

Confirm PyTorch and Torchvision versions below:

In [ ]:
import torch
print(torch.__version__)

import os
import scipy
import yaml
print(yaml.__version__)
print(scipy.__version__)

1.2.0
5.2
1.1.0


Clone AlphaPose repository, which will allow us to install it and run locally.

In [ ]:
# remove existing directory for the repo (if exists)
!rm -rf /content/AlphaPose
os.chdir('/content/')

# official version still has issues with newer versions of PyTorch
# to overcome we can merge pull request 592 on repo (see further below)
# if using PyTorch 1.4 or older, don't worry about this - just clone the repo
!git clone https://github.com/MVIG-SJTU/AlphaPose.git

# use modified version that supports updated versions of PyTorch & Torchvision
#!git clone https://github.com/RodrigoGantier/AlphaPose.git

Cloning into 'AlphaPose'...
remote: Enumerating objects: 2278, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 2278 (delta 7), reused 0 (delta 0), pack-reused 2245
Receiving objects: 100% (2278/2278), 108.47 MiB | 33.95 MiB/s, done.
Resolving deltas: 100% (1108/1108), done.


There is a recent error with one of the commits made in Jan-2022, and so we need to revert to the previous version of the repo so that the models work. We'll do this below:

In [ ]:
os.chdir('/content/AlphaPose')
print(os.getcwd())

/content/AlphaPose


In [ ]:
!git config --global user.email "example@hotmail.co.uk"
!git config --global user.name "Example user"

In [ ]:
!git reset --hard bcfbc997526bcac464d116356ac2efea9483ff68

HEAD is now at bcfbc99 Update ShuffleResnet.py


To use PyTorch versions 1.5+, we need to merge pull request 592 on the AlphaPose repository:

In [ ]:
# if using PyTorch 1.5+, merge pull request 592, as below:
#!git config --global user.email "example@hotmail.co.uk"
#!git config --global user.name "Example user"
#!git pull origin pull/592/head

fatal: not a git repository (or any of the parent directories): .git


We also need to ensure we have cython installed (for c-speed efficiency), and libyaml-dev:

In [ ]:
!python -m pip install cython
!sudo apt-get install libyaml-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libyaml-dev is already the newest version (0.1.7-2ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


We can now build AlphaPose in the local instance, as below:

In [ ]:
os.chdir('/content/AlphaPose')
print(os.getcwd())

/content/AlphaPose


In [ ]:
# if not already done so - build alphapose
! python setup.py build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/trackers
copying trackers/tracker_cfg.py -> build/lib.linux-x86_64-3.7/trackers
copying trackers/tracker_api.py -> build/lib.linux-x86_64-3.7/trackers
copying trackers/__init__.py -> build/lib.linux-x86_64-3.7/trackers
creating build/lib.linux-x86_64-3.7/alphapose
copying alphapose/version.py -> build/lib.linux-x86_64-3.7/alphapose
copying alphapose/opt.py -> build/lib.linux-x86_64-3.7/alphapose
copying alphapose/__init__.py -> build/lib.linux-x86_64-3.7/alphapose
creating build/lib.linux-x86_64-3.7/trackers/tracking
copying trackers/tracking/basetrack.py -> build/lib.linux-x86_64-3.7/trackers/tracking
copying trackers/tracking/matching.py -> build/lib.linux-x86_64-3.7/trackers/tracking
copying trackers/tracking/__init__.py -> build/lib.linux-x86_64-3.7/trackers/tracking
creating build/lib.linux-x86_64-3.7/trackers/ReidModels
copying trackers/ReidModels/net_utils.py -> 

## 3. Downloading Pre-trained Models and Detector Weights

We need to download the pre-trained weights and models to use AlphaPose, which we'll do below after authenticating Colab.

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Download pre-trained weights and models - takes a few minutes.

In [ ]:
! mkdir /content/AlphaPose/detector/yolo/data
file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/detector/yolo/data/yolov3-spp.weights')

! mkdir /content/AlphaPose/detector/tracker/data
file_id = '1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/detector/tracker/data/JDE-1088x608-uncertainty')

file_id = '1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/pretrained_models/fast_res50_256x192.pth')

## 4. Upload custom dataset into newly created folders

In [ ]:
!mkdir input_images
!mkdir output_images

mkdir: cannot create directory ‘input_images’: File exists


Upload desired dataset of images to the `AlphaPose/input_images` folder. You need to run the cell above to create this directory initially if you haven't already done so. 

To upload files to the directory, you can use the Colab file explorer / navigation menu on the left. Right click the input_images folder, click upload, and select all of the desired images to upload them.

Once done, you should see the uploaded files listed below:

In [ ]:
!ls input_images

 example-input.png     train_1008.png	     train_1017.png   train_1040.png
 train_0.png	       train_1009.png	     train_1018.png   train_1048.png
 train_1000.png        train_100.png	     train_1019.png   train_1065.png
 train_1001.png       'train_1010 (1).png'   train_101.png    train_1084.png
 train_1002.png        train_1010.png	     train_1020.png   train_10.png
 train_1003.png        train_1011.png	     train_1021.png   train_1109.png
'train_1004 (1).png'   train_1012.png	     train_1022.png   train_1120.png
 train_1004.png        train_1013.png	     train_1023.png   train_1121.png
 train_1005.png        train_1014.png	     train_102.png    train_1130.png
 train_1006.png        train_1015.png	     train_1035.png   train_1.png
 train_1007.png        train_1016.png	     train_1037.png


## 5. Inference on directory of images using AlphaPose

We can now perform inference to obtain pose features for all people in each scene within our dataset, like so:

In [ ]:
# perform inference using pre-trained model on images listed within the demo folder
!python3 scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml --checkpoint pretrained_models/fast_res50_256x192.pth --indir input_images --outdir output_images --save_img

Loading YOLO model..
Loading pose model from pretrained_models/fast_res50_256x192.pth...
100% 43/43 [00:07<00:00,  5.57it/s]
===========================> Finish Model Running.
===========================> Rendering remaining images in the queue...
===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).
===========================> Rendering remaining 36 images in the queue...
===========================> Rendering remaining 35 images in the queue...
===========================> Rendering remaining 32 images in the queue...
===========================> Rendering remaining 30 images in the queue...
===========================> Rendering remaining 27 images in the queue...
===========================> Rendering remaining 22 images in the queue...
===========================> Rendering remaining 20 images in the queue...
===========================> Rendering remaining 18 images in the queue...
========================

Our results from this can be viewed within the examples/res folder:

In [ ]:
# result json and rendered images are saved here:
!ls output_images
!ls output_images/vis

alphapose-results.json	vis
 example-input.png     train_1008.png	     train_1017.png   train_1040.png
 train_0.png	       train_1009.png	     train_1018.png   train_1048.png
 train_1000.png        train_100.png	     train_1019.png   train_1065.png
 train_1001.png       'train_1010 (1).png'   train_101.png    train_1084.png
 train_1002.png        train_1010.png	     train_1020.png   train_10.png
 train_1003.png        train_1011.png	     train_1021.png   train_1109.png
'train_1004 (1).png'   train_1012.png	     train_1022.png   train_1120.png
 train_1004.png        train_1013.png	     train_1023.png   train_1121.png
 train_1005.png        train_1014.png	     train_102.png    train_1130.png
 train_1006.png        train_1015.png	     train_1035.png   train_1.png
 train_1007.png        train_1016.png	     train_1037.png


## 6. Save results and download

Lets zip these results and download them accordingly.

In [ ]:
os.getcwd()

'/content/AlphaPose'

In [ ]:
!zip -r /content/results.zip /content/AlphaPose/output_images

  adding: content/AlphaPose/output_images/ (stored 0%)
  adding: content/AlphaPose/output_images/alphapose-results.json (deflated 65%)
  adding: content/AlphaPose/output_images/vis/ (stored 0%)
  adding: content/AlphaPose/output_images/vis/train_1006.png (deflated 3%)
  adding: content/AlphaPose/output_images/vis/train_1013.png (deflated 5%)
  adding: content/AlphaPose/output_images/vis/train_1084.png (deflated 3%)
  adding: content/AlphaPose/output_images/vis/train_1130.png (deflated 4%)
  adding: content/AlphaPose/output_images/vis/example-input.png (deflated 6%)
  adding: content/AlphaPose/output_images/vis/train_1010.png (deflated 6%)
  adding: content/AlphaPose/output_images/vis/train_1019.png (deflated 2%)
  adding: content/AlphaPose/output_images/vis/train_1015.png (deflated 5%)
  adding: content/AlphaPose/output_images/vis/train_1.png (deflated 3%)
  adding: content/AlphaPose/output_images/vis/train_1000.png (deflated 2%)
  adding: content/AlphaPose/output_images/vis/train_1023

In [ ]:
from google.colab import files
files.download("/content/results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

You should now have all the results, including json annotations and visualisations of pose features on original images, as a downloaded zip file.